In [ ]:
import pandas as pd
import numpy as np
import os

player = "Kaylan Bigun"
path = "Shot_Visuals_KaylanBigun_Combined.csv"

output_dir = os.path.join(os.getcwd())
os.makedirs(output_dir, exist_ok=True)

In [ ]:
events = pd.read_csv(path)
events['pointWonBy'] = events.groupby('pointNumber')['pointWonBy'].bfill()
returns_ucla = events[(events['shotHitBy'] == player) & (events['shotInRally'] == 2)][['pointStartTime', 'shotHitBy', 'shotContactX', 'shotContactY', 'pointWonBy', 'isWinner', 'shotFhBh']].dropna(subset=['pointWonBy', 'shotContactX', 'shotContactY']).copy()
returns_ucla['shotContactX'] = returns_ucla.apply(lambda row: -row['shotContactX'] if row['shotContactY'] > 0 else row['shotContactX'], axis=1)
returns_ucla['shotContactY'] = returns_ucla['shotContactY'].apply(lambda y: -y if y > 0 else y)
returns_ucla['depth'] = returns_ucla['shotContactY'].apply(
    lambda y: 'short' if y >= -455 else 'mid' if -455 > y > -490 else 'deep'
)

In [ ]:
def return_place(player, path, side, fh_bh, exclude_times=None):
    # Load the data
    events = pd.read_csv(path)
    events['pointWonBy'] = events.groupby('pointNumber')['pointWonBy'].bfill()
    events['isError'] = (events['isErrorWideR'] == 1) | (events['isErrorWideL'] == 1) | (events['isErrorNet'] == 1) | (events['isErrorLong'] == 1)
    
    # Filter for the player's returns and shots in rally
    returns_place = events[(events['shotHitBy'] == player) & (events['shotInRally'] == 2)][
        ['pointStartTime', 'shotHitBy', 'shotContactX', 'shotContactY', 'shotLocationX', 'shotLocationY',
         'pointWonBy', 'isWinner', 'shotFhBh', 'isError', 'isErrorNet', 'side']
    ].dropna(subset=['pointWonBy']).copy()
    
    # If exclude_times is provided, filter out those rows
    if exclude_times:
        returns_place = returns_place[~returns_place['pointStartTime'].isin(exclude_times)]

    # Flip shotContactX and shotContactY where necessary
    mask_bottom_half = (returns_place['shotLocationY'] < 0) & (returns_place['shotContactY'] > 0)
    mask_near_net = (returns_place['shotLocationY'] <= 50) & (returns_place['shotContactY'] > 0) & (returns_place['isErrorNet'] == 1)

    returns_place.loc[mask_bottom_half, 'shotContactX'] *= -1
    returns_place.loc[mask_bottom_half, 'shotLocationX'] *= -1
    returns_place.loc[mask_bottom_half & (returns_place['shotContactY'] > 0), 'shotContactY'] *= -1
    returns_place.loc[mask_bottom_half, 'shotLocationY'] = returns_place.loc[mask_bottom_half, 'shotLocationY'].abs()

    returns_place.loc[mask_near_net & ~mask_bottom_half, 'shotContactX'] *= -1
    returns_place.loc[mask_near_net & ~mask_bottom_half, 'shotLocationX'] *= -1
    returns_place.loc[mask_near_net & ~mask_bottom_half, 'shotContactY'] *= -1

    # Accounting for net error tagging discrepencies
    mask = (returns_place['shotLocationY'] != 0) & (returns_place['isErrorNet'] == 1)
    adjust_up = mask & (returns_place['shotLocationX'] <= returns_place['shotContactX'])
    adjust_down = mask & (returns_place['shotLocationX'] > returns_place['shotContactX'])

    returns_place.loc[adjust_up, 'shotLocationX'] += returns_place.loc[adjust_up, 'shotLocationY']
    returns_place.loc[adjust_down, 'shotLocationX'] -= returns_place.loc[adjust_down, 'shotLocationY']

    returns_place.loc[adjust_up, 'shotContactX'] += returns_place.loc[adjust_up, 'shotLocationY']
    returns_place.loc[adjust_down, 'shotContactX'] -= returns_place.loc[adjust_down, 'shotLocationY']
    returns_place.loc[adjust_up, 'shotLocationY'] = 0
    returns_place.loc[adjust_down, 'shotLocationY'] = 0

    # Additional filtering for fh_bh and/or side is specified
    if side != 'All':
        if fh_bh != 'All':
            returns_place = returns_place[returns_place['shotFhBh'] == fh_bh]
        returns_place = returns_place[returns_place['side'] == side]

    returns_place['fhBhFiltered'] = [fh_bh != 'All'] * len(returns_place)
    returns_place['sideFiltered'] = [side != 'All'] * len(returns_place)

    # Categorize into 'left', 'mid', 'right' based on shotLocationX
    returns_place['width'] = returns_place['shotLocationX'].apply(
        lambda x: 'left' if x <= -52.5 else 'mid' if -52.5 < x < 52.5 else 'right'
    )

    # Calculate count + win pct.
    distribution = returns_place.groupby('width').apply(
        lambda df: pd.Series({
            'freq': len(df),
            'win_percentage': int((df['pointWonBy'] == df['shotHitBy']).mean() * 100)
        })
    ).reset_index()

    max_win_percentage = distribution['win_percentage'].max()
    min_win_percentage = distribution['win_percentage'].min()

    # Assign 'max', 'min', or 'no' to the distribution based on win_percentage
    distribution['maxMin'] = distribution['win_percentage'].apply(
        lambda x: 'max' if x == max_win_percentage else 'min' if x == min_win_percentage else 'no'
    )

    # Convert win_percentage to string for display
    distribution['win_percentage'] = distribution['win_percentage'].astype(str) + '%'

    # Adjust x_mapping to match the width values
    x_mapping = {
        'left': {'x': -100},
        'mid': {'x': 0},
        'right': {'x': 100}
    }

    # Export the data as JSON
    return_place_json = returns_place.to_json(orient='records')
    return_place_dist_json = distribution.to_json(orient='records')

    with open(os.path.join(output_dir, 'return_place.json'), 'w') as f:
        f.write(return_place_json)

    with open(os.path.join(output_dir, 'return_place_dist.json'), 'w') as f:
        f.write(return_place_dist_json)
